In [ ]:
import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
import torchvision
import matplotlib.pyplot as plt
import os
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import transforms, datasets, models

In [ ]:
#修改图片为3*224*224
data_transform = transforms.Compose([transforms.RandomResizedCrop(224),transforms.RandomHorizontalFlip(),transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
#从data/train读入训练集
train_img = torchvision.datasets.ImageFolder('data/train',
                                            transform=data_transform
                                            )
test_img = torchvision.datasets.ImageFolder('data/val',
                                            transform=data_transform
                                            )
#从data/val读入测试集
train_data = torch.utils.data.DataLoader(train_img, batch_size=4,shuffle=True)
test_data = torch.utils.data.DataLoader(test_img, batch_size=4,shuffle=True)

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3,16,kernel_size=5,padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16,32,kernel_size=2,padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Linear(64,2)
#卷积+卷积+全连接
        
#前向传播
    def forward(self,x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = out.view(out.size(0),-1)
        return out


net = Net()

NameError: name 'nn' is not defined

In [ ]:
# 模型加载到cpu或gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device',device)
net.to(device)

In [ ]:
#代价函数与优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr=0.001,momentum=0.9)

# 训练网络
for epoch in range(2):
    running_loss = 0.0
    for i,data in enumerate(train_data,0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

#统计信息
        running_loss += loss.item()
        if i % 200 ==199:
            print('[%d,%5d] loss:%.3f'%(epoch + 1,i+1,running_loss /2000))
            running_loss = 0.0

print('Finished Training')

In [ ]:
#保存模型
PATH_checkpoint = './data/model.pth'
torch.save(net.state_dict(),PATH_checkpoint)

In [ ]:
# 加载模型
net_predict = Net()
net_predict.load_state_dict(torch.load(PATH_checkpoint))

#测试集验证
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in test_data:
        images, labels = data
        outputs = net_predict(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(3):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

classes = ('cuke','lettuce','lotus_root'  )

for i in range(3):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))